# Setting more informative priors

## Preliminaries

In [4]:
library(brms)

Loading required package: Rcpp

Loading 'brms' package (version 2.18.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').


Attaching package: ‘brms’


The following object is masked from ‘package:stats’:

    ar




In [9]:
N <- 100
M <- 2
K <- 4
J <- 3

mu <- 0
al <- matrix(c(-1, 1), ncol = 1)
Z <- matrix(rnorm(M*N, sd = .5), ncol = N)
ep_x <- matrix(rnorm(N, sd = 0.1), ncol = N)
X <- t(al) %*% Z + ep_x

U <- matrix(rnorm(K*N, sd = .2), ncol = N)
W <- sweep(U, 2, X, "+")

h_1 <- function(x) 0
h_2 <- function(x) 0
f <- function(x) 2*x
eta <- rnorm(N)
S <- f(X) + h_1(Z[1, ]) + h_2(Z[2, ]) + eta
S <- matrix(S, ncol = N)

beta0 <- matrix(rep(0, J*N), ncol = N)
beta1 <- matrix(rep(1, J), ncol = 1)
ep_y <- matrix(rnorm(J * N, sd = 0.1), ncol = N)
Y <- beta0 + beta1 %*% S + ep_y

In [10]:
data <- as.data.frame(t(rbind(Z,X,W,Y)))
names(data) <- c("Z1", "Z2", "X","W1", "W2", "W3", "W4", "Y1", "Y2", "Y3")
data$S <- as.numeric(NA)


## Simple Model

In [11]:
bform <- bf(X ~ 0 + mi(S)) + bf(S|mi() ~ 0) + bf(Y1 ~ 0 + mi(S)) + set_rescor(FALSE)
fit <- brm(bform, data = data)

Compiling Stan program...

Start sampling




SAMPLING FOR MODEL 'anon_model' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 5e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.5 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 0.42 seconds (Warm-up)
Chain 1:                0.251 seconds (Sampling)
Chain 1:                0.671 se

Warning message:
“There were 127 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is 3.01, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains f

In [12]:
summary(fit)

Warning message:
“Parts of the model have not converged (some Rhats are > 1.05). Be careful when analysing the results! We recommend running more iterations and/or setting stronger priors.”
Warning message:
“There were 127 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(gaussian, gaussian, gaussian) 
  Links: mu = identity; sigma = identity
         mu = identity; sigma = identity
         mu = identity; sigma = identity 
Formula: X ~ 0 + mi(S) 
         S | mi() ~ 0 
         Y1 ~ 0 + mi(S) 
   Data: data (Number of observations: 100) 
  Draws: 4 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup draws = 4000

Population-Level Effects: 
       Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
X_miS    -16.99     40.56  -131.95    26.49 3.01        5       16
Y1_miS   -40.54     95.40  -305.96    58.10 3.01        5       14

Family Specific Parameters: 
         Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sigma_X      0.33      0.11     0.08     0.49 1.10       46       43
sigma_S      0.14      0.17     0.01     0.61 2.34        5       23
sigma_Y1     0.77      0.24     0.28     1.15 1.07       68       92

Draws were sampled using sampling(NUTS). For each parameter, Bulk_ESS
and

In [13]:
prior_summary(fit)

prior,class,coef,group,resp,dpar,nlpar,lb,ub,source
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
,b,,,X,,,,,default
,b,miS,,X,,,,,default
,b,,,Y1,,,,,default
,b,miS,,Y1,,,,,default
"student_t(3, 0, 2.5)",sigma,,,S,,,0,,default
"student_t(3, 0, 2.5)",sigma,,,X,,,0,,default
"student_t(3, 0, 2.5)",sigma,,,Y1,,,0,,default


In [15]:
priors <- c(
    prior(normal(3,1), class = b, coef = miS, resp = X),
    prior(normal(1,1), class = b, coef = miS, resp = Y1)
)

fit <- brm(bform, data = data, prior = priors)

Compiling Stan program...

Start sampling




SAMPLING FOR MODEL 'anon_model' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 5.7e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.57 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 0.467 seconds (Warm-up)
Chain 1:                0.827 seconds (Sampling)
Chain 1:                1.29

Warning message:
“There were 47 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is 1.31, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains fo

In [16]:
summary(fit)

Warning message:
“Parts of the model have not converged (some Rhats are > 1.05). Be careful when analysing the results! We recommend running more iterations and/or setting stronger priors.”
Warning message:
“There were 47 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(gaussian, gaussian, gaussian) 
  Links: mu = identity; sigma = identity
         mu = identity; sigma = identity
         mu = identity; sigma = identity 
Formula: X ~ 0 + mi(S) 
         S | mi() ~ 0 
         Y1 ~ 0 + mi(S) 
   Data: data (Number of observations: 100) 
  Draws: 4 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup draws = 4000

Population-Level Effects: 
       Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
X_miS      1.21      0.48     0.29     2.13 1.22       14       12
Y1_miS     2.46      0.84     0.69     4.03 1.18       16       12

Family Specific Parameters: 
         Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sigma_X      0.26      0.11     0.07     0.47 1.12       25       63
sigma_S      0.71      0.45     0.33     2.23 1.20       15       12
sigma_Y1     0.90      0.20     0.39     1.16 1.05       69      160

Draws were sampled using sampling(NUTS). For each parameter, Bulk_ESS
and

In [17]:
priors <- c(
    prior(normal(3,.1), class = b, coef = miS, resp = X),
    prior(normal(1,.1), class = b, coef = miS, resp = Y1)
)

fit <- brm(bform, data = data, prior = priors)

Compiling Stan program...

Start sampling




SAMPLING FOR MODEL 'anon_model' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 6.7e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.67 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 0.194 seconds (Warm-up)
Chain 1:                0.509 seconds (Sampling)
Chain 1:                0.70

Warning message:
“There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is 1.19, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#tail-ess”


In [18]:
summary(fit)

Warning message:
“Parts of the model have not converged (some Rhats are > 1.05). Be careful when analysing the results! We recommend running more iterations and/or setting stronger priors.”


 Family: MV(gaussian, gaussian, gaussian) 
  Links: mu = identity; sigma = identity
         mu = identity; sigma = identity
         mu = identity; sigma = identity 
Formula: X ~ 0 + mi(S) 
         S | mi() ~ 0 
         Y1 ~ 0 + mi(S) 
   Data: data (Number of observations: 100) 
  Draws: 4 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup draws = 4000

Population-Level Effects: 
       Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
X_miS      2.95      0.10     2.75     3.15 1.01      548      946
Y1_miS     1.11      0.10     0.92     1.32 1.00      686      952

Family Specific Parameters: 
         Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sigma_X      0.15      0.10     0.02     0.38 1.12       24       48
sigma_S      0.25      0.02     0.21     0.29 1.01      516     1167
sigma_Y1     1.52      0.11     1.33     1.76 1.00      800     1949

Draws were sampled using sampling(NUTS). For each parameter, Bulk_ESS
and

## Full model, but linear

In [21]:
N <- 100
M <- 2
K <- 4
J <- 3

mu <- 0
al <- matrix(c(-1, 1), ncol = 1)
Z <- matrix(rnorm(M*N, sd = .5), ncol = N)
ep_x <- matrix(rnorm(N, sd = 0.1), ncol = N)
X <- t(al) %*% Z + ep_x

U <- matrix(rnorm(K*N, sd = .2), ncol = N)
W <- sweep(U, 2, X, "+")

h_1 <- function(x) -x
h_2 <- function(x) 3*x
f <- function(x) 2*x
eta <- rnorm(N)
S <- f(X) + h_1(Z[1, ]) + h_2(Z[2, ]) + eta
S <- matrix(S, ncol = N)

beta0 <- matrix(rep(0, J*N), ncol = N)
beta1 <- matrix(rep(1, J), ncol = 1)
ep_y <- matrix(rnorm(J * N, sd = 0.1), ncol = N)
Y <- beta0 + beta1 %*% S + ep_y

In [27]:
data$Xm <- as.numeric(NA)

bf2 <- bf(S|mi() ~ mi(Xm) + Z1 + Z2)
bf3 <- bf(Y1 ~ 0 + mi(S))
bf4 <- bf(Y2 ~ 0 + mi(S))
bf5 <- bf(Y3 ~ 0 + mi(S))
bf6 <- bf(Z1 ~ 0 + mi(Xm))
bf7 <- bf(Z2 ~ 0 + mi(Xm))
bf8 <- bf(Xm|mi() ~ 0)

bf9 <- bf(Y1 ~ Z1+Z2 + mi(Xm)+mi(S))

model <- brm(bf2 + bf3 + bf4 + bf5 + bf6 + bf7 + bf8 +set_rescor(FALSE), data = data)

Compiling Stan program...

Start sampling




SAMPLING FOR MODEL 'anon_model' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0.000105 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 1.05 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 6.62 seconds (Warm-up)
Chain 1:                11.15 seconds (Sampling)
Chain 1:                17.7

Warning message:
“There were 562 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 1413 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 10. See
https://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded”
Warning message:
“There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is 2.49, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
ht

In [28]:
summary(model)

Warning message:
“Parts of the model have not converged (some Rhats are > 1.05). Be careful when analysing the results! We recommend running more iterations and/or setting stronger priors.”
Warning message:
“There were 562 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(gaussian, gaussian, gaussian, gaussian, gaussian, gaussian, gaussian) 
  Links: mu = identity; sigma = identity
         mu = identity; sigma = identity
         mu = identity; sigma = identity
         mu = identity; sigma = identity
         mu = identity; sigma = identity
         mu = identity; sigma = identity
         mu = identity; sigma = identity 
Formula: S | mi() ~ mi(Xm) + Z1 + Z2 
         Y1 ~ 0 + mi(S) 
         Y2 ~ 0 + mi(S) 
         Y3 ~ 0 + mi(S) 
         Z1 ~ 0 + mi(Xm) 
         Z2 ~ 0 + mi(Xm) 
         Xm | mi() ~ 0 
   Data: data (Number of observations: 100) 
  Draws: 4 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup draws = 4000

Population-Level Effects: 
             Estimate Est.Error   l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
S_Intercept      0.19      0.57      -0.68     1.51 1.34       10      110
S_Z1             2.81      6.76      -6.81    15.02 1.98        6       63
S_Z2            -2.07      6.73    

In [29]:
prior_summary(model)

prior,class,coef,group,resp,dpar,nlpar,lb,ub,source
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"student_t(3, 0, 2.5)",Intercept,,,S,,,,,default
,b,,,S,,,,,default
,b,Z1,,S,,,,,default
,b,Z2,,S,,,,,default
,b,miXm,,S,,,,,default
,b,,,Y1,,,,,default
,b,miS,,Y1,,,,,default
,b,,,Y2,,,,,default
,b,miS,,Y2,,,,,default


In [31]:
priors <- c(
    prior(normal(-1, 1), class = b, coef = Z1, resp = S),
    prior(normal(3, 1), class = b, coef = Z2, resp = S),
    prior(normal(0, 1), class = b, coef = miXm, resp = S),
    prior(normal(1, 1), class = b, coef = miS, resp = Y1),
    prior(normal(1, 1), class = b, coef = miS, resp = Y2),
    prior(normal(1, 1), class = b, coef = miS, resp = Y3),
    prior(normal(-1, 1), class = b, coef = miXm, resp = Z1),
    prior(normal(1, 1), class = b, coef = miXm, resp = Z2)
)

model <- brm(bf2 + bf3 + bf4 + bf5 + bf6 + bf7 + bf8 +set_rescor(FALSE), data = data, prior = priors)

Compiling Stan program...

Start sampling




SAMPLING FOR MODEL 'anon_model' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0.000127 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 1.27 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 5.493 seconds (Warm-up)
Chain 1:                6.903 seconds (Sampling)
Chain 1:                12.

Warning message:
“There were 986 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 277 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 10. See
https://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded”
Warning message:
“There were 3 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is 2.24, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
htt

In [32]:
summary(model)

Warning message:
“Parts of the model have not converged (some Rhats are > 1.05). Be careful when analysing the results! We recommend running more iterations and/or setting stronger priors.”
Warning message:
“There were 986 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(gaussian, gaussian, gaussian, gaussian, gaussian, gaussian, gaussian) 
  Links: mu = identity; sigma = identity
         mu = identity; sigma = identity
         mu = identity; sigma = identity
         mu = identity; sigma = identity
         mu = identity; sigma = identity
         mu = identity; sigma = identity
         mu = identity; sigma = identity 
Formula: S | mi() ~ mi(Xm) + Z1 + Z2 
         Y1 ~ 0 + mi(S) 
         Y2 ~ 0 + mi(S) 
         Y3 ~ 0 + mi(S) 
         Z1 ~ 0 + mi(Xm) 
         Z2 ~ 0 + mi(Xm) 
         Xm | mi() ~ 0 
   Data: data (Number of observations: 100) 
  Draws: 4 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup draws = 4000

Population-Level Effects: 
            Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
S_Intercept     0.05      0.18    -0.35     0.40 1.43        8       31
S_Z1            0.78      1.72    -2.88     2.82 2.01        6       37
S_Z2            1.73      1.23    -0.05     

In [33]:
priors <- c(
    prior(normal(-1, .1), class = b, coef = Z1, resp = S),
    prior(normal(3, .1), class = b, coef = Z2, resp = S),
    prior(normal(0, .1), class = b, coef = miXm, resp = S),
    prior(normal(1, .1), class = b, coef = miS, resp = Y1),
    prior(normal(1, .1), class = b, coef = miS, resp = Y2),
    prior(normal(1, .1), class = b, coef = miS, resp = Y3),
    prior(normal(-1, .1), class = b, coef = miXm, resp = Z1),
    prior(normal(1, .1), class = b, coef = miXm, resp = Z2)
)

model <- brm(bf2 + bf3 + bf4 + bf5 + bf6 + bf7 + bf8 +set_rescor(FALSE), data = data, prior = priors)

Compiling Stan program...

Start sampling




SAMPLING FOR MODEL 'anon_model' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0.000119 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 1.19 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 1.701 seconds (Warm-up)
Chain 1:                1.496 seconds (Sampling)
Chain 1:                3.1

Warning message:
“There were 1 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is 1.13, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for

In [34]:
summary(model)$fixed

Warning message:
“Parts of the model have not converged (some Rhats are > 1.05). Be careful when analysing the results! We recommend running more iterations and/or setting stronger priors.”
Warning message:
“There were 1 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


,Estimate,Est.Error,l-95% CI,u-95% CI,Rhat,Bulk_ESS,Tail_ESS
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
S_Intercept,-0.1757661296,0.13930969,-0.4441348,0.1003517,1.001002,4323.82640,1882.0427
S_Z1,-1.1150504237,0.09480844,-1.2985332,-0.9320417,1.000859,4702.44978,2466.1177
S_Z2,2.8778693594,0.09978830,2.6750300,3.0706836,1.001620,2593.77683,1687.3579
S_miXm,0.0004406981,0.09725450,-0.1862563,0.1908271,1.001539,4532.40082,2650.1977
Y1_miS,0.9259501037,0.04512503,0.8407304,1.0178528,1.039999,85.89351,191.6594
Y2_miS,0.9188863537,0.04506247,0.8344672,1.0120520,1.039456,87.85233,269.5959
Y3_miS,0.9294615951,0.04529852,0.8441146,1.0223252,1.040190,85.69370,238.3877
Z1_miXm,-0.9927617810,0.10305031,-1.2002699,-0.7869071,1.002301,2628.61483,1909.4443
Z2_miXm,0.9940223048,0.09915843,0.7984717,1.1817605,1.001710,3873.35173,3014.1738


In [37]:
summary(model)$spec_pars

Warning message:
“Parts of the model have not converged (some Rhats are > 1.05). Be careful when analysing the results! We recommend running more iterations and/or setting stronger priors.”
Warning message:
“There were 1 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


,Estimate,Est.Error,l-95% CI,u-95% CI,Rhat,Bulk_ESS,Tail_ESS
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
sigma_S,1.33413050,0.11189470,1.13576580,1.5760446,1.007696,468.74676,1291.18482
sigma_Y1,0.09664608,0.01227873,0.07241084,0.1209848,1.003977,1397.77169,1527.54867
sigma_Y2,0.12232609,0.01173712,0.10058782,0.1473152,1.000257,2589.44648,2152.02288
sigma_Y3,0.09028654,0.01227888,0.06600359,0.1147027,1.002112,1369.80106,1697.98776
sigma_Z1,0.49452111,0.04095892,0.41818009,0.5806906,1.005532,800.95236,2008.70433
sigma_Z2,0.48571454,0.04137838,0.40686129,0.5700967,1.007887,479.49614,1525.92975
sigma_Xm,0.13040434,0.07132711,0.01455318,0.2744768,1.122439,23.15566,24.94241


In [39]:
priors <- c(
    prior(normal(-1, 1), class = b, coef = Z1, resp = S),
    prior(normal(3, 1), class = b, coef = Z2, resp = S),
    prior(normal(0, 1), class = b, coef = miXm, resp = S),
    prior(normal(1, 1), class = b, coef = miS, resp = Y1),
    prior(normal(1, 1), class = b, coef = miS, resp = Y2),
    prior(normal(1, 1), class = b, coef = miS, resp = Y3),
    prior(normal(-1, 1), class = b, coef = miXm, resp = Z1),
    prior(normal(1, 1), class = b, coef = miXm, resp = Z2)
)

model <- brm(bf2 + bf3 + bf4 + bf5 + bf6 + bf7 + bf8 +set_rescor(FALSE), data = data, prior = priors, iter = 10000)

Compiling Stan program...

Start sampling




SAMPLING FOR MODEL 'anon_model' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0.000133 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 1.33 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 10000 [  0%]  (Warmup)
Chain 1: Iteration: 1000 / 10000 [ 10%]  (Warmup)
Chain 1: Iteration: 2000 / 10000 [ 20%]  (Warmup)
Chain 1: Iteration: 3000 / 10000 [ 30%]  (Warmup)
Chain 1: Iteration: 4000 / 10000 [ 40%]  (Warmup)
Chain 1: Iteration: 5000 / 10000 [ 50%]  (Warmup)
Chain 1: Iteration: 5001 / 10000 [ 50%]  (Sampling)
Chain 1: Iteration: 6000 / 10000 [ 60%]  (Sampling)
Chain 1: Iteration: 7000 / 10000 [ 70%]  (Sampling)
Chain 1: Iteration: 8000 / 10000 [ 80%]  (Sampling)
Chain 1: Iteration: 9000 / 10000 [ 90%]  (Sampling)
Chain 1: Iteration: 10000 / 10000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 22.093 seconds (Warm-up)
Chain 1:                32.888 seconds (Sampling)
Chain 1:    

Warning message:
“There were 664 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 153 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 10. See
https://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded”
Warning message:
“There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is 1.68, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
htt

In [40]:
summary(model)$fixed

Warning message:
“Parts of the model have not converged (some Rhats are > 1.05). Be careful when analysing the results! We recommend running more iterations and/or setting stronger priors.”
Warning message:
“There were 664 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


,Estimate,Est.Error,l-95% CI,u-95% CI,Rhat,Bulk_ESS,Tail_ESS
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
S_Intercept,0.06905141,0.1724986,-0.2825742,0.3999082,1.315267,9.827193,32.09461
S_Z1,0.89869333,1.6579175,-2.5793405,2.7331050,1.543981,7.284443,32.36529
S_Z2,2.03673246,1.0634827,-0.4130513,4.0731939,1.037770,728.170964,302.97747
S_miXm,0.44247218,1.3807218,-2.4038597,2.4651162,1.510760,7.269149,31.24461
Y1_miS,-0.42802128,0.8647176,-1.3842493,1.3977658,1.556024,7.324673,35.54158
Y2_miS,-0.42474178,0.8581258,-1.3718103,1.3876980,1.556021,7.324992,34.33362
Y3_miS,-0.42963902,0.8680734,-1.3893906,1.4037978,1.556055,7.322983,34.79631
Z1_miXm,-0.19071023,0.5768238,-2.0377848,0.2981241,1.380525,12.931001,42.24953
Z2_miXm,0.08172883,0.6001508,-0.5748421,1.8376512,1.396198,8.427494,34.18448


In [41]:
save(list = ls(), file = "priors_brms.RData")